In [ ]:
!pip -q install -U autoawq transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, AutoConfig
import gc
import os

/usr/local/lib/python3.12/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes

In [ ]:
# Configure environment
os.environ["TOKENIZERS_PARALLELISM"] = "false"
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print("Checking PyTorch and CUDA versions...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name()}")

Checking PyTorch and CUDA versions...
PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA version: 12.6
GPU: Tesla T4


In [ ]:
# Clear any existing cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

180

In [ ]:
print("Loading tokenizer...")
tok = AutoTokenizer.from_pretrained(
    base_model,
    use_fast=True,
    trust_remote_code=True
)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
print("Loading model...")
# Load model with specific configurations to avoid attention issues
mdl = AutoAWQForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    use_cache=False,
    trust_remote_code=True,
    torch_dtype=torch.float16,  # Use fp16 explicitly
    device_map={"": 0} if torch.cuda.is_available() else "cpu"  # More specific device mapping
)

Loading model...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

eval_results.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
# Set model to eval mode
mdl.eval()

LlamaAWQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 2048)
      (layers): ModuleList(
        (0-21): 22 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear(in_features=2048, out_features=256, bias=False)
            (v_proj): Linear(in_features=2048, out_features=256, bias=False)
            (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
            (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
            (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        )

In [ ]:
# AWQ quantization config
quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}

In [ ]:
# Create more diverse and shorter calibration data
print("Preparing calibration data...")
calib_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning models process data efficiently.",
    "Natural language understanding is advancing rapidly.",
    "Deep neural networks learn complex patterns.",
    "Artificial intelligence transforms technology.",
    "Computer vision recognizes objects accurately.",
    "Robotics integrates sensors and actuators.",
    "Algorithm optimization improves performance significantly.",
    "Data science extracts meaningful insights.",
    "Software engineering creates reliable systems."
] * 10  # 100 samples total

Preparing calibration data...


In [ ]:
import os
os.environ["PYTORCH_USE_SDPA"] = "0"

In [ ]:
from transformers import AutoConfig
cfg = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
cfg.attn_implementation = "eager"   # fallback attention

In [ ]:
calib_tokens = [
    tok(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128).input_ids
    for text in calib_texts[:50]
]


In [ ]:
mdl.quantize(
    tok,
    quant_config=quant_config,
    calib_data=calib_tokens,
    max_calib_seq_len=128,
    max_calib_samples=50,
    n_parallel_calib_samples=1
)


NotImplementedError: Either pass a string to a huggingface dataset or a listthat is preprocessed with one sample of text per element or a list of list of int for tokenized words.

In [ ]:
import os
os.environ["PYTORCH_USE_SDPA"] = "0"   # Disable SDPA attention

calib_tokens = [
    tok(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128).input_ids
    for text in calib_texts[:50]
]

mdl.quantize(
    tok,
    quant_config=quant_config,
    calib_data=calib_tokens,
    max_calib_seq_len=128,
    max_calib_samples=50,
    n_parallel_calib_samples=1
)


NotImplementedError: Either pass a string to a huggingface dataset or a listthat is preprocessed with one sample of text per element or a list of list of int for tokenized words.

In [ ]:
calib_tokens = [
    tok(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128).input_ids[0].tolist()
    for text in calib_texts[:50]
]

mdl.quantize(
    tok,
    quant_config=quant_config,
    calib_data=calib_tokens,   # now proper format
    max_calib_seq_len=128,
    max_calib_samples=50,
    n_parallel_calib_samples=1
)


AWQ:   0%|          | 0/22 [00:00<?, ?it/s]


RuntimeError: The expanded size of the tensor (1) must match the existing size (50) at non-singleton dimension 0.  Target sizes: [1, 32, 128, 128].  Tensor sizes: [50, 1, 128, 128]

In [ ]:
mdl.quantize(
    tok,
    quant_config=quant_config,
    calib_data=calib_texts[:50],   # raw text list
    max_calib_seq_len=128,
    max_calib_samples=50,
    n_parallel_calib_samples=1
)

AWQ:   0%|          | 0/22 [00:00<?, ?it/s]


RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [1, 32, 128, 128].  Tensor sizes: [3, 1, 128, 128]

In [ ]:
# Save the quantized model
out_dir = "tinyllama-1.1b-awq"
print(f"Saving model to {out_dir}...")

mdl.save_quantized(out_dir, safetensors=True)
tok.save_pretrained(out_dir)

# Save config
config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
config.save_pretrained(out_dir)

print(f"Model successfully quantized and saved to {out_dir}")

AWQ:   0%|          | 0/22 [00:00<?, ?it/s]


RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [1, 32, 128, 128].  Tensor sizes: [3, 1, 128, 128]

In [ ]:
print("Starting quantization...")
try:
    # Use minimal configuration to avoid batch size issues
    mdl.quantize(
        tok,
        quant_config=quant_config,
        calib_data=calib_texts,
        max_calib_seq_len=128,     # Reduced sequence length
        max_calib_samples=50,      # Reduced sample count
        n_parallel_calib_samples=1 # Keep sequential processing
    )

    print("Quantization completed successfully!")

    # Save the quantized model
    out_dir = "tinyllama-1.1b-awq"
    print(f"Saving model to {out_dir}...")

    mdl.save_quantized(out_dir, safetensors=True)
    tok.save_pretrained(out_dir)

    # Save config
    config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
    config.save_pretrained(out_dir)

    print(f"Model successfully quantized and saved to {out_dir}")

except Exception as e:
    print(f"Quantization failed with error: {str(e)}")
    print("Trying alternative approach...")
    print("\nTrying alternative approach with different model loading...")

    # Alternative approach: Load model differently
    del mdl
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    try:
        # Try loading without device_map first
        mdl = AutoAWQForCausalLM.from_pretrained(
            base_model,
            low_cpu_mem_usage=True,
            trust_remote_code=True,
            use_cache=False
        )

        # Move to device manually if CUDA is available
        if torch.cuda.is_available():
            mdl = mdl.cuda()

        mdl.eval()

        # Try with even smaller calibration parameters
        print("Attempting quantization with minimal parameters...")
        mdl.quantize(
            tok,
            quant_config=quant_config,
            calib_data=calib_texts[:20],  # Only use first 20 samples
            max_calib_seq_len=64,         # Even smaller sequence length
            max_calib_samples=20,         # Minimal samples
            n_parallel_calib_samples=1
        )

        out_dir = "tinyllama-1.1b-awq"
        mdl.save_quantized(out_dir, safetensors=True)
        tok.save_pretrained(out_dir)

        config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
        config.save_pretrained(out_dir)

        print(f"Model successfully quantized with alternative approach and saved to {out_dir}")

    except Exception as e2:
        print(f"Alternative approach also failed: {str(e2)}")
        print("\nConsider using a pre-quantized model instead:")
        print("TheBloke/TinyLlama-1.1B-Chat-v1.0-AWQ")

finally:
    # Cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

/usr/local/lib/python3.11/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


Checking PyTorch and CUDA versions...
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU: Tesla T4
Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Quantization failed with error: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [1, 32, 128, 128].  Tensor sizes: [3, 1, 128, 128]

Trying alternative approach with different model loading...


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Attempting quantization with minimal parameters...


AWQ:   0%|          | 0/22 [00:00<?, ?it/s]


Alternative approach also failed: output with shape [1, 32, 64, 64] doesn't match the broadcast shape [2, 32, 64, 64]

Consider using a pre-quantized model instead:
TheBloke/TinyLlama-1.1B-Chat-v1.0-AWQ


The script runs an AWQ quantization pipeline with a safe, low-memory fallback path.

Primary flow:

Calls mdl.quantize(...) using your tokenizer, AWQ quant config, and calibration texts (prompts).

Uses conservative settings to avoid OOM: max_calib_seq_len=128, max_calib_samples=50, n_parallel_calib_samples=1.

On success, saves the quantized weights, tokenizer, and model config to out_dir.

If quantization fails:

Frees memory (del, torch.cuda.empty_cache(), gc.collect()).

Reloads the model with low_cpu_mem_usage=True and use_cache=False (and moves to GPU if available).

Retries quantization with smaller calibration settings (shorter seq length, fewer samples).

If it still fails, suggests using a pre-quantized AWQ model.

Role of prompts (calibration texts):

They provide short, representative inputs so AWQ can observe activations and pick good weight scales.

More and more realistic prompts ⇒ better quality after quantization; your code uses a small set to keep memory low.

Important parameters:

max_calib_seq_len & max_calib_samples: control calibration length/size (quality vs. memory).

n_parallel_calib_samples: set to 1 for minimal VRAM use.

low_cpu_mem_usage=True, use_cache=False: reduce RAM/GPU memory footprint during load/quant.

safetensors=True, trust_remote_code=True: safer format and allow custom model code.

Gotchas / prerequisites:

Ensure variables & imports exist: mdl, tok, calib_texts, quant_config, base_model, plus torch, gc, AutoConfig, and AutoAWQForCausalLM (from the AWQ library).

Don’t mix GPTQ configs with AWQ; use the right quant config for the backend.

If you hit OOM, reduce max_calib_seq_len/samples further or increase group size.

After quantization:

Load the saved model directory with the AWQ loader (from_quantized) and run generation as usual.